In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import pandas as pd

# Set up Selenium WebDriver with WebDriver Manager
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run in headless mode (optional)
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--window-size=1920,1080")

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

# Open the website
url = "https://pft.hyrox.com/ranking/global"
driver.get(url)
time.sleep(5)  # Wait for page to load

all_data = []


# Function to extract data from the page
def extract_data():
    soup = BeautifulSoup(driver.page_source, "html.parser")
    rows = soup.find_all("tr", class_="focus-within:bg-gray-lighter")

    for row in rows:
        cols = row.find_all("td")
        if len(cols) >= 6:
            rank = cols[0].text.strip()
            name = cols[1].text.strip()
            time = cols[2].text.strip()
            location = cols[3].text.strip()
            age_group = cols[4].text.strip()
            medal = cols[5].find("img")["alt"] if cols[5].find("img") else "None"

            all_data.append([rank, name, time, location, age_group, medal])


# Extract data from all pages
while True:
    extract_data()
    try:
        # Locate the "Next page" button and click it
        next_button = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable(
                (
                    By.XPATH,
                    "//a[contains(@class, 'cursor-pointer') and .//span[contains(text(), 'Next page')]]",
                )
            )
        )
        next_button.click()
        time.sleep(3)  # Wait for new data to load
    except:
        print("No more pages left.")
        break

# Close the driver
driver.quit()

# Save data to CSV
df = pd.DataFrame(
    all_data, columns=["Rank", "Name", "Time", "Location", "Age Group", "Medal"]
)
df.to_csv("hyrox_rankings.csv", index=False)
print("Data saved to hyrox_rankings.csv ✅")

No more pages left.
Data saved to hyrox_rankings.csv ✅
